# Fit teaching models to behavioral data
Natalia Vélez & Alicia Chen, November 2021

In [1]:
import sys, pprint
import numpy as np
import pandas as pd
from ast import literal_eval as eval_tuple
from scipy.stats import entropy
from scipy.ndimage import convolve

sys.path.append('..')
from utils import read_json, write_json, int_extract

Load teaching problems:

In [2]:
problems = read_json('inputs/problems.json')
problems[0]

{'A': [[0, 0, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0],
  [1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1],
  [0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 0, 0]],
 'B': [[1, 1, 1, 0, 0, 0],
  [1, 1, 1, 0, 0, 0],
  [1, 1, 1, 1, 0, 0],
  [0, 0, 1, 1, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0]],
 'C': [[0, 0, 0, 1, 1, 1],
  [0, 0, 0, 1, 1, 1],
  [0, 0, 1, 1, 1, 1],
  [0, 0, 1, 1, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0]],
 'D': [[0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 1, 1, 0, 0],
  [1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1]]}

Load exclusions:

In [3]:
excluded = np.loadtxt('../1_preprocessing/outputs/excluded_participants.txt', dtype=str)
excluded = [int_extract('[0-9]+', s) for s in excluded]
print(excluded)

[3, 17]


Load teaching data:

In [4]:
def read_examples(e):
    coords = eval_tuple(e)
    idx = np.ravel_multi_index(coords, (6,6))
    
    return idx

human_df = pd.read_csv('outputs/teaching_behavior.csv')
human_df = human_df.drop(columns=['onset', 'order', 'rating']) # drop columns that are irrelevant for model
human_df = human_df[~human_df.subject.isin(excluded)] # exclude wiggly participants
human_df = human_df[~pd.isna(human_df.example)] # exclude trials where teachers failed to respond
human_df['example'] = human_df.example.apply(read_examples)

human_df.head()

,subject,run,block_idx,ex_idx,problem,example
0,1,1,0,0,22,8
1,1,1,0,1,22,27
3,1,1,1,0,18,3
4,1,1,1,1,18,32
5,1,1,1,2,18,7


## Helper functions

Convert teaching problem (dict) into dataframe of coordinates x hypotheses

In [5]:
def hypotheses_dataframe(hypotheses):
    '''
    general method: converts (4,6,6) array (used, e.g., in teaching problems, priors, etc.) 
    into an examples x hypotheses dataframe
    '''
    
    hypotheses_flat = np.reshape(hypotheses, (hypotheses.shape[0], hypotheses.shape[1]*hypotheses.shape[2])) #  flatten 3d => 2d array

    ### reshape into dataframe of coordinates x hypotheses
    df = pd.DataFrame(hypotheses_flat).stack().rename_axis(['hypothesis', 'idx']).reset_index(name='val')
    
    # name hypotheses
    df['hypothesis'] = pd.Categorical(df.hypothesis)
    df['hypothesis'] = df.hypothesis.cat.rename_categories(['A', 'B', 'C', 'D'])
    df['hypothesis'] = df['hypothesis'].astype('object')
    
    # spread each hypothesis into its own column
    df = df.pivot(index='idx', columns='hypothesis', values='val')
    df = df[df.sum(axis=1) > 0]
    
    return df

def problem_df(prob_idx):
    '''
    reads problem and converts to dataframe
    '''
    prob = problems[prob_idx]
    hypotheses = np.array(list(prob.values())) # read hypothesis space
    df = hypotheses_dataframe(hypotheses)
    
    return df

In [6]:
problem_df(20) # let's test it out!

hypothesis,A,B,C,D
idx,,,,
9,1,1,1,0
10,0,1,0,0
14,1,1,1,0
15,1,1,1,0
16,0,1,0,0
19,1,1,1,1
20,1,1,1,1
21,1,1,1,1
22,0,1,0,0


Returns a matrix of examples x hypotheses, where each entry indicates whether a given hypothesis is consistent with this next example *and all examples that came before it*

In [7]:
def filter_consistent_examples(prob_idx, past_examples=[]):
    example_space = problem_df(prob_idx)
    possible_examples = example_space.copy()
    
    for ex in past_examples:
        consistent_with_past = possible_examples.loc[ex] # which hypotheses did this hypothesis rule out?
        possible_examples = possible_examples.drop(ex) # drop past examples from consideration
        
        # drop hypotheses that are incompatible with this past example
        possible_examples = possible_examples.mul(consistent_with_past, axis=1)
        possible_examples = possible_examples[possible_examples.columns[possible_examples.sum()>0]]
        
    return possible_examples

In [30]:
generate_edge_prior(22, [8]) # let's test it out!

hypothesis,A,B,D
idx,,,
1,0.076923,0.074468,0.000000
2,0.000000,0.000000,0.089552
3,0.000000,0.000000,0.089552
4,0.076923,0.074468,0.000000
7,0.065934,0.063830,0.000000
9,0.054945,0.053191,0.074627
10,0.065934,0.063830,0.000000
13,0.065934,0.063830,0.104478
16,0.065934,0.063830,0.104478


Return full belief distribution (used to generate model predictions)

In [39]:
def full_belief(nonzero_belief):
    belief = nonzero_belief.reindex(['A', 'B', 'C', 'D'], fill_value=0)
    return belief

Convert a pandas dataseries to a list of tuples containing `(index, value)` (we're going to use this to save model predictions to JSON files)

In [40]:
def series2tuple(s):
    return list(zip(bs.index,s))

## Sampling methods

### a) Strong sampling

Define the sampling method:

In [33]:
def strong_sampling(prob_idx, past_examples=[]):
    '''
    Input: Index of problem (as it appears in "problems" list)
    Output: Dataframe of the probability of selecting the data (idx), given the hypothesis (A, B, C, D)
    '''
    # find available examples
    available_examples = filter_consistent_examples(prob_idx, past_examples=past_examples)

    # select uniformly among available examples
    pD = available_examples.div(available_examples.sum(axis=0), axis=1)
    
    return pD

Main method: Compute likelihood of data under strong sampling

In [34]:
def fit_strong_sampling(group):
    model_outputs = []
    examples = []
    
    # initialize belief distribution
    # (uniform prior over hypotheses)
    belief = np.ones(4)*.25
    belief_in_true = belief[0]
    
    for _, row in group.iterrows():
        ex = row.example
        # likelihood of observed data, assuming strong sampling
        strong_pD = strong_sampling(row.problem, past_examples=examples)
        out = row.copy()
        out['model'] = 'strong'
        out['lik'] = strong_pD['A'].loc[ex] # likelihood of selected example
        out['pD'] = series2tuple(strong_pD['A']) # full sampling distribution
        
        # learner's posterior belief given the data
        strong_pH = strong_pD.div(strong_pD.sum(axis=1), axis=0)
        new_belief = full_belief(strong_pH.loc[ex])
        out['pTrue'] = strong_pH['A'].loc[ex] # probability of true hypotheses
        out['pH'] = series2tuple(new_belief) # full belief distribution
        out['entropy'] = entropy(new_belief.values) # entropy of belief distribution
        
        # change in beliefs
        out['delta'] = new_belief['A'] - belief_in_true
        out['KL'] = entropy(new_belief.values, belief)
        belief = new_belief.values # change values for next round
        belief_in_true = belief[0]
        
        out = out.to_dict()
        examples.append(ex)
        model_outputs.append(out)
    
    return model_outputs

Loop through behavioral data:

In [14]:
strong_predictions = []
for name, group in human_df.groupby(['subject', 'run', 'block_idx']):
    pred = fit_strong_sampling(group)
    strong_predictions += pred
    
pprint.pprint(strong_predictions[0])

{'KL': 0.2998411459755585,
 'block_idx': 0,
 'delta': 0.05538922155688625,
 'entropy': 1.0864532151443322,
 'ex_idx': 0,
 'example': 8,
 'lik': 0.0625,
 'model': 'strong',
 'pD': [(1, 0.0625),
        (2, 0.0),
        (3, 0.0),
        (4, 0.0625),
        (7, 0.0625),
        (8, 0.0625),
        (9, 0.0625),
        (10, 0.0625),
        (13, 0.0625),
        (16, 0.0625),
        (19, 0.0625),
        (22, 0.0625),
        (25, 0.0625),
        (26, 0.0625),
        (27, 0.0625),
        (28, 0.0625),
        (31, 0.0625),
        (32, 0.0),
        (33, 0.0),
        (34, 0.0625)],
 'pH': [('A', 0.30538922155688625),
        ('B', 0.2874251497005988),
        ('C', 0.0),
        ('D', 0.40718562874251496)],
 'pTrue': 0.30538922155688625,
 'problem': 22,
 'run': 1,
 'subject': 1}


Save model predictions to file

In [15]:
write_json(strong_predictions, 'outputs/model_predictions_strong.json')

### b) Spatial bias model

Define the sampling method:

In [35]:
def spatially_biased_sampling(prob_idx, past_examples=[]):
    '''
    This spatially-biased method places greater weight on selecting examples that 
    have fewer neighbors (i.e., an "edge prior")
    '''    
    # read hypothesis space
    prob = problems[prob_idx]
    hypothesis_space = np.array(list(prob.values()))

    # assign weight to each square based on number of empty spaces
    kernel = np.array([
        [1,1,1],
        [1,0,1],
        [1,1,1]
    ]) # convolution kernel (used to count # of neighbors)

    n_neighbors = np.zeros(hypothesis_space.shape)
    for i in range(hypothesis_space.shape[0]):
        n_neighbors[i,:,:] = convolve(hypothesis_space[i,:,:], kernel, mode='constant')
    edge_weight = 8-n_neighbors
    edge_weight = edge_weight+1 # don't entirely rule out "landlocked" squares    

    # keep only examples that are consistent with the ones that came before
    all_squares = hypotheses_dataframe(edge_weight)
    valid_examples = filter_consistent_examples(prob_idx, past_examples)
    edge_df = all_squares[all_squares.index.isin(valid_examples.index)].copy()
    edge_df = valid_examples.mul(edge_df[valid_examples.columns])
    
    # normalize
    edge_df = edge_df.div(edge_df.sum(axis=0), axis=1)
    return edge_df

Generate spatial bias predictions:

In [36]:
def fit_spatial_bias(group):
    model_outputs = []
    examples = []
    
    # initialize belief distribution
    # (uniform prior over hypotheses)
    belief = np.ones(4)*.25
    belief_in_true = belief[0]
    
    for _, row in group.iterrows():
        ex = row.example
        # likelihood of observed data, assuming strong sampling
        bias_pD = spatially_biased_sampling(row.problem, past_examples=examples)
        out = row.copy()
        out['model'] = 'spatial_bias'
        out['lik'] = bias_pD['A'].loc[ex] # likelihood of selected example
        out['pD'] = series2tuple(bias_pD['A']) # full sampling distribution
        
        # learner's posterior belief given the data
        bias_pH = bias_pD.div(bias_pD.sum(axis=1), axis=0)
        new_belief = full_belief(bias_pH.loc[ex])
        out['pTrue'] = bias_pH['A'].loc[ex] # probability of true hypotheses
        out['pH'] = series2tuple(new_belief) # full belief distribution
        out['entropy'] = entropy(new_belief.values) # entropy of belief distribution
        
        # change in beliefs
        out['delta'] = new_belief['A'] - belief_in_true
        out['KL'] = entropy(new_belief.values, belief)
        belief = new_belief.values # change values for next round
        belief_in_true = belief[0]
        
        out = out.to_dict()
        examples.append(ex)
        model_outputs.append(out)
    
    return model_outputs

Loop through behavioral data:

In [37]:
biased_predictions = []
for name, group in human_df.groupby(['subject', 'run', 'block_idx']):
    pred = fit_spatial_bias(group)
    biased_predictions += pred
    
pprint.pprint(biased_predictions[0])

{'KL': 0.2985487019218349,
 'block_idx': 0,
 'delta': 0.05275229357798167,
 'entropy': 1.0877456591980557,
 'ex_idx': 0,
 'example': 8,
 'lik': 0.052083333333333336,
 'model': 'spatial_bias',
 'pD': [(1, 0.07291666666666667),
        (2, 0.0),
        (3, 0.0),
        (4, 0.07291666666666667),
        (7, 0.0625),
        (8, 0.052083333333333336),
        (9, 0.052083333333333336),
        (10, 0.0625),
        (13, 0.0625),
        (16, 0.0625),
        (19, 0.0625),
        (22, 0.0625),
        (25, 0.0625),
        (26, 0.052083333333333336),
        (27, 0.052083333333333336),
        (28, 0.0625),
        (31, 0.07291666666666667),
        (32, 0.0),
        (33, 0.0),
        (34, 0.07291666666666667)],
 'pH': [('A', 0.30275229357798167),
        ('B', 0.2935779816513761),
        ('C', 0.0),
        ('D', 0.4036697247706422)],
 'pTrue': 0.30275229357798167,
 'problem': 22,
 'run': 1,
 'subject': 1}


In [38]:
write_json(biased_predictions, 'outputs/model_predictions_spatialbias.json')

### c) Pedagogical sampling

Define the belief updating method:

In [33]:
def pedagogical_belief_updating(prob_idx, past_examples, last_pH):
    '''
    In sequential cooperative Bayesian inference (Wang, Wang & Shafto, 2020), 
    the posterior on the last trial is then passed on as the prior for the next trial. 
    
    Or, in other words... "yesterday's posterior is tomorrow's prior"
    This function then updates the learner's beliefs according to:
    P(H|D) = P(H)P(D|H) / P(D)
    '''
    # drop past examples (can't be repeated)
    pD = filter_consistent_examples(prob_idx, past_examples=past_examples)
    pD = pD.div(pD.sum(axis=0), axis=1) # normalize columns
    
    # update pH
    last_example = past_examples[-1]
    yesterdays_posterior = last_pH.loc[last_example]
    prior_unnorm = pD.mul(yesterdays_posterior, axis=1)
    tomorrows_prior = prior_unnorm.div(prior_unnorm.sum(axis=1), axis=0)
    
    # drop hypotheses that are incompatible with past examples
    # (makes some of the math nicer - otherwise we'll run into divide-by-0 errors in Sinkhorn scaling)
    tomorrows_prior = tomorrows_prior[tomorrows_prior.columns[tomorrows_prior.sum()>0]]
    
    return tomorrows_prior

Define the sampling method:

In [46]:
def pedagogical_sampling(prob_idx, past_examples=[], last_pH=None, nIter=10):
        
    if len(past_examples):
        pH = pedagogical_belief_updating(prob_idx, past_examples, last_pH)
    # else: start from a uniform prior
    else:
        hypothesis_space = problem_df(prob_idx)
        uniform_prior = hypothesis_space.div(hypothesis_space.sum(axis=1), axis=0)
        pH = uniform_prior
        
    # ~ recursive reasoning ~
    for _ in range(nIter):
        pD = pH.div(pH.sum(axis=0), axis=1)
        pH = pD.div(pD.sum(axis=1), axis=0)
        
    return pD, pH

Main method: Compute likelihood of data under pedagogical sampling

In [47]:
def fit_pedagogical_sampling(group, nIter=20):
    model_outputs = []
    
    # initialize inputs to sampler
    examples = []
    pH = None
    pD = None
    
    # initialize belief distribution
    # (uniform prior over hypotheses)
    belief = np.ones(4)*.25
    belief_in_true = belief[0]
    
    for _, row in group.iterrows():
        ex = row.example
        out = row.copy()
        out['model'] = 'pedagogical'

        # likelihood of observed data, assuming pedagogical sampling
        pD, pH = pedagogical_sampling(row.problem, past_examples=examples, last_pH=pH, nIter=nIter)
        out['lik'] = pD['A'].loc[ex]
        out['pD'] = series2tuple(pD['A']) # full sampling distribution
        
        # learner's posterior belief given the data
        new_belief = full_belief(pH.loc[ex])
        out['pTrue'] = new_belief['A']
        out['pH'] = series2tuple(new_belief) # full belief distribution
        out['entropy'] = entropy(new_belief.values) # entropy of belief distribution
        
        # change in beliefs
        out['delta'] = new_belief['A'] - belief_in_true
        out['KL'] = entropy(new_belief.values, belief)
        belief = new_belief.values # change values for next round
        belief_in_true = belief[0]
        
        out = out.to_dict()
        examples.append(ex)
        model_outputs.append(out)
        
    return model_outputs

Loop through behavioral data:

In [48]:
pedagogical_predictions = []
nIter=10
for name, group in human_df.groupby(['subject', 'run', 'block_idx']):
    pred = fit_pedagogical_sampling(group, nIter=nIter)
    pedagogical_predictions += pred
    
pprint.pprint(pedagogical_predictions[0])

{'KL': 0.2973894774766346,
 'block_idx': 0,
 'delta': 0.06966637719365176,
 'entropy': 1.0889048836432562,
 'ex_idx': 0,
 'example': 8,
 'lik': 0.06393328805971461,
 'model': 'pedagogical',
 'pD': [(1, 0.07050694086392065),
        (2, 0.0),
        (3, 0.0),
        (4, 0.07050694086392065),
        (7, 0.07050694086392065),
        (8, 0.06393328805971461),
        (9, 0.06393328805971461),
        (10, 0.07050694086392065),
        (13, 0.04906615817640507),
        (16, 0.04906615817640507),
        (19, 0.04906615817640507),
        (22, 0.04906615817640507),
        (25, 0.07050694086392065),
        (26, 0.04906615817640507),
        (27, 0.06274710608717998),
        (28, 0.07050694086392065),
        (31, 0.07050694086392065),
        (32, 0.0),
        (33, 0.0),
        (34, 0.07050694086392065)],
 'pH': [('A', 0.31966637719365176),
        ('B', 0.28409742653588654),
        ('C', 0.0),
        ('D', 0.3962361962704616)],
 'pTrue': 0.31966637719365176,
 'problem': 22,
 'run

Save model_predictions to file

In [19]:
write_json(pedagogical_predictions, 'outputs/model_predictions_pedagogical_n%i.json' % nIter)

### d) Pedagogical sampling + spatial bias

Belief updating:

In [41]:
def spatial_pedagogical_update(prob_idx, past_examples, last_pH):
    '''
    In sequential cooperative Bayesian inference (Wang, Wang & Shafto, 2020), 
    the posterior on the last trial is then passed on as the prior for the next trial. 
    
    Or, in other words... "yesterday's posterior is tomorrow's prior"
    This function then updates the learner's beliefs according to:
    P(H|D) = P(H)P(D|H) / P(D)
    
    This function differs from the pure pedagogical belief updating in that
    p(d|h) is grounded in a spatially biased prior, rather than a uniform one
    '''
    # drop past examples (can't be repeated)
    pD = spatially_biased_sampling(prob_idx, past_examples)
    
    # update pH
    last_example = past_examples[-1]
    yesterdays_posterior = last_pH.loc[last_example]
    prior_unnorm = pD.mul(yesterdays_posterior, axis=1)
    tomorrows_prior = prior_unnorm.div(prior_unnorm.sum(axis=1), axis=0)
    
    # drop hypotheses that are incompatible with past examples
    # (makes some of the math nicer - otherwise we'll run into divide-by-0 errors in Sinkhorn scaling)
    tomorrows_prior = tomorrows_prior[tomorrows_prior.columns[tomorrows_prior.sum()>0]]
    
    return tomorrows_prior

Define the sampling method:

In [46]:
def spatial_pedagogical_sampling(prob_idx, past_examples=[], last_pH=None, nIter=10):
        
    if len(past_examples):
        pH = spatial_pedagogical_update(prob_idx, past_examples, last_pH)
    # else: start from a spatially biased prior
    else:
        pD = spatially_biased_sampling(prob_idx)
        pH = pD.div(pD.sum(axis=1), axis=0)
        
    # ~ recursive reasoning ~
    for _ in range(nIter):
        pD = pH.div(pH.sum(axis=0), axis=1)
        pH = pD.div(pD.sum(axis=1), axis=0)
        
    return pD, pH

Fit to behavioral data:

In [47]:
def fit_spatial_pedagogical_sampling(group, nIter=20):
    model_outputs = []
    
    # initialize inputs to sampler
    examples = []
    pH = None
    pD = None
    
    # initialize belief distribution
    # (uniform prior over hypotheses)
    belief = np.ones(4)*.25
    belief_in_true = belief[0]
    
    for _, row in group.iterrows():
        ex = row.example
        out = row.copy()
        out['model'] = 'pedagogical_plus_spatial_bias'

        # likelihood of observed data, assuming pedagogical sampling
        pD, pH = spatial_pedagogical_sampling(row.problem, past_examples=examples, last_pH=pH, nIter=nIter)
        out['lik'] = pD['A'].loc[ex]
        out['pD'] = series2tuple(pD['A']) # full sampling distribution
        
        # learner's posterior belief given the data
        new_belief = full_belief(pH.loc[ex])
        out['pTrue'] = new_belief['A']
        out['pH'] = series2tuple(new_belief) # full belief distribution
        out['entropy'] = entropy(new_belief.values) # entropy of belief distribution
        
        # change in beliefs
        out['delta'] = new_belief['A'] - belief_in_true
        out['KL'] = entropy(new_belief.values, belief)
        belief = new_belief.values # change values for next round
        belief_in_true = belief[0]
        
        out = out.to_dict()
        examples.append(ex)
        model_outputs.append(out)
        
    return model_outputs

Loop through behavioral data:

In [49]:
spatial_pedagogical_predictions = []
nIter=10
for name, group in human_df.groupby(['subject', 'run', 'block_idx']):
    pred = fit_spatial_pedagogical_sampling(group, nIter=nIter)
    spatial_pedagogical_predictions += pred
    
pprint.pprint(spatial_pedagogical_predictions[0])

{'KL': 0.29226158189876716,
 'block_idx': 0,
 'delta': 0.0741031974293132,
 'entropy': 1.0940327792211237,
 'ex_idx': 0,
 'example': 8,
 'lik': 0.06482063340722197,
 'model': 'pedagogical_plus_spatial_bias',
 'pD': [(1, 0.07011358450770017),
        (2, 0.0),
        (3, 0.0),
        (4, 0.07011358450770017),
        (7, 0.0701135845077002),
        (8, 0.06482063340722197),
        (9, 0.06482063340722197),
        (10, 0.0701135845077002),
        (13, 0.04584825312310762),
        (16, 0.04584825312310762),
        (19, 0.04753448667267756),
        (22, 0.04585600269502843),
        (25, 0.07411506653800348),
        (26, 0.052231634711429796),
        (27, 0.0647278405889353),
        (28, 0.0701135845077002),
        (31, 0.07351568868706516),
        (32, 0.0),
        (33, 0.0),
        (34, 0.07011358450770017)],
 'pH': [('A', 0.3241031974293132),
        ('B', 0.2994861975027223),
        ('C', 0.0),
        ('D', 0.3764106050679644)],
 'pTrue': 0.3241031974293132,
 'problem

Save model predictions to file

In [50]:
write_json(spatial_pedagogical_predictions, 'outputs/model_predictions_spatialpedagogical_n%i.json' % nIter)